<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/Python/11.1.Asthma_Mortality_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌳 Predicting Asthma Mortality Rate using Random Forest



Random Forest (RF) is an ensemble learning method that combines the output of multiple individual decision trees to make more accurate and stable predictions. It can be used for both classification (predicting categories) and regression (predicting continuous numbers) tasks

This notebook explores the use of  RF  to predict the Normalized Asthma Mortality Rate (NAMR) across departments in Argentina. By leveraging historical time series data from 2001 to 2021 and incorporating relevant environmental and demographic covariates, the objective is to estimate asthma mortality rate for 2022 and evaluate model performance using multiple error metrics.


## 📚 Import required libraries

In [ ]:
# dataframe libraries
import pandas as pd
import numpy as np

# geospatial libraries
import geopandas as gpd
import mapclassify
from libpysal.weights import Queen
from esda.moran import Moran
from pysal.explore import esda

# plot libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.patches import Patch
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,explained_variance_score,median_absolute_error, max_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# other libraries
import os
from joblib import Parallel, delayed
import shap
from itables import init_notebook_mode, show

In [ ]:
%cd work/

In [ ]:
# Set the PROJ_LIB path
os.environ['PROJ_LIB'] = "/opt/conda/envs/gds/share/proj"

In [ ]:
 # Copy-on-Write
 pd.options.mode.copy_on_write = True

##🗂 Load and transform the dataset

In [ ]:
# Load dataset with data per department
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/data.gpkg")

In [ ]:
# Reshape df to ts long format
years = range(2001, 2023)
records = []

In [ ]:
# drop geometry and PDPM25
for _, row in gdf.iterrows():
    iddpto = row["IDDPTO"]
    for year in years:
        records.append({
            "IDDPTO": iddpto,
            "YEAR": year,
            "CA": row.get(f"CA_{year}", np.nan),
            "PM25": row.get(f"PM25_{year}", np.nan),
            "NBA": row.get(f"NBA_{year}", np.nan),
            "PD": row.get(f"PD_{year}", np.nan),
            "NAGRT": row.get(f"NAGRT_{year}", np.nan),
            "NNWVT": row.get(f"NNWVT_{year}", np.nan),
            "NBUT": row.get(f"NBUT_{year}", np.nan),
            "ELEV": row.get(f"ELEV_{year}", np.nan)
            })

In [ ]:
# create new df from list
df_ts = pd.DataFrame(records)

In [ ]:
# Create lag variables (up to 2 years)
def create_lags(df, var, max_lag=2):
    for lag in range(1, max_lag+1):
        df[f"{var}_lag{lag}"] = df[var].shift(lag)
    return df

In [ ]:
for var in ["NAGRT", "NNWVT", "NBUT"]:
    df_ts = create_lags(df_ts, var)

In [ ]:
# Drop the initial rows with NaNs due to lagging
df_ts = df_ts.dropna().reset_index(drop=True)

In [ ]:
# visualize the dataframe
init_notebook_mode(all_interactive=True)
show(df_ts)

##📈 Random Forest modelling

In this section we will train a RF model using data from 2001 to 2002 to predict NAMR (CA) in 2022

### Split train  and test sets

In [ ]:
# Split train (2001–2021) and test (2022)
train = df_ts[(df_ts['YEAR'] >= 2001) & (df_ts['YEAR'] <= 2021)]
test  = df_ts[df_ts['YEAR'] == 2022]

In [ ]:
 # Select features (exclude IDDPTO, YEAR, and CA)
exclude_cols = ['IDDPTO', 'YEAR', 'CA']
features = [c for c in df_ts.columns if c not in exclude_cols]

X_train = train[features]
y_train = train['CA']
X_test  = test[features]

In [ ]:
# visualize feautes
init_notebook_mode(all_interactive=True)
X_train.head()

In [ ]:
# visualize feautes
init_notebook_mode(all_interactive=True)
y_train.head()

In [ ]:
# visualize feautes
init_notebook_mode(all_interactive=True)
X_test.head()

### Training RF

In [ ]:
# Train Random Forest with default hiperparameters
rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1,
)
rf.fit(X_train, y_train)

### Predicting NAMR for 2022

In [ ]:
# Predict CA for 2022
test['CA_2022_pred'] = rf.predict(X_test)

In [ ]:
# Output predictions
pred_2022 = test[['IDDPTO', 'YEAR', "CA", 'CA_2022_pred']]
# rename "CA" to "CA_2022"
pred_2022.rename(columns={'CA': 'CA_2022'}, inplace=True)

In [ ]:
# visualize pred_2022
init_notebook_mode(all_interactive=True)
show(pred_2022)

### Evaluating model performance

In [ ]:
# True vs Predicted vectors
y_true = pred_2022['CA_2022'].values
y_pred = pred_2022['CA_2022_pred'].values

In [ ]:
# Metrics
MAE  = mean_absolute_error(y_true, y_pred)
RMSE = np.sqrt(mean_squared_error(y_true, y_pred))
R2 = r2_score(y_true, y_pred)

In [ ]:
print("Model Performance on 2022 Prediction")
print("------------------------------------")
print(f"MAE  : {MAE:.4f}")
print(f"RMSE : {RMSE:.4f}")
print(f"R²   : {R2:.4f}")

In [ ]:
# create a scatterplot between CA_2022 vs CA_2022_Pred
plt.figure(figsize=(10, 6))
plt.scatter(pred_2022['CA_2022'], pred_2022['CA_2022_pred'])
plt.xlabel('CA_2022 (Actual)')
plt.ylabel('CA_2022_Pred (Predicted)')
plt.title('Scatter Plot of Actual vs. Predicted Mortality Rates (2022)')
plt.grid(True)
plt.show()

The evaluation metrics indicate that the Random Forest model has weak predictive performance for estimating asthma mortality rates in 2022.

* A MAE of 1.61 and RMSE of 2.21 show that prediction errors are relatively large compared to the scale of the target variable.

* The negative R² (-0.08) indicates that the model performs worse than simply predicting the mean of CA for all departments.

The scatter plot confirms this: predictions cluster in a narrow range (mostly between 1 and 2), while actual values span a much wider range, including many zeros and several high counts. This suggests that the model fails to capture variability, especially the zero-inflation and right tail, leading to underestimation of high values and overestimation of zeros.

Overall, the model struggles with the distributional characteristics of the data, indicating the need for zero-inflated, two-stage (hurdle) models or alternative approaches better suited for sparse count-like outcomes.

##  🚧 Hurdle-type RF modelling

Hurdle-type modeling is a statistical approach designed to analyze count data, particularly when there is an excess of zeros. This model operates in two stages: the first stage determines whether the count is zero or positive using a binary classification, while the second stage models only the positive counts, often employing a regression models.[1]

### Part 1 – RF Classification Model

In this section, we will train and evaluate a Random Forest (RF) classification model to predict whether the Normalized Asthma Mortality Rate (NAMR, represented by the variable CA) indicates the nonoccurrence (0) or occurrence (1) of asthma mortality (binary classification). We will apply a walk-forward (expanding window) validation approach, which is appropriate for epidemiological studies involving time series data.[2] We will start by training the RF classification model using data from 2001 to 2006 (a 5-year window) and testing it with data from 2007. The training window will then be expanded by one year at each iteration until it spans from 2001 to 2021, with 2022 data used for testing.


#### Loading and transforming the data

In [ ]:
# Binary target
df_ts['CA_bin'] = (df_ts['CA'] > 0).astype(int)

In [ ]:
# visusalize the dataframe
init_notebook_mode(all_interactive=True)
df_ts.head()

#### Training and Testing a RF Classification Model

In [ ]:
results = []  # results list

for i in range(2006, 2022):  # start walk-forward from year 2006 to 2021
    train_years = list(range(2001, i + 1))  # expanding window
    test_year = i + 1
    print(f"Training: {train_years[0]}–{train_years[-1]}, Testing: {test_year}")

    # Split train/test by year
    train_df = df_ts[df_ts['YEAR'].isin(train_years)]
    test_df = df_ts[df_ts['YEAR'] == test_year]

    # Define features and target
    features = ['PM25', 'NBA', 'PD', 'NAGRT', 'NNWVT', 'NBUT', 'ELEV',
              'NAGRT_lag1', 'NAGRT_lag2', 'NNWVT_lag1', 'NNWVT_lag2',
              'NBUT_lag1','NBUT_lag2']
    target = 'CA_bin'

    X_train = train_df[features]
    y_train = train_df[target]
    X_test = test_df[features]
    y_test = test_df[target]

    # Train classification model with default hiperparameters
    model = RandomForestClassifier(n_jobs=-1, random_state=42)
    model.fit(X_train, y_train)

    # Predict (labels)
    y_pred = model.predict(X_test)

    # Classification metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    # Store results
    results.append({
        'train_years': f"{train_years[0]}-{train_years[-1]}",
        'test_year': test_year,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1_Score': f1
    })

In [ ]:
# convert results into a dataframe
results_df = pd.DataFrame(results)

In [ ]:
# visualize the results
init_notebook_mode(all_interactive=True)
results_df

In [ ]:
# Calculate and display the mean and standard deviation for each evaluation metric
mean_metrics = results_df[['Accuracy', 'Precision', 'Recall', 'F1_Score']].mean()
std_metrics = results_df[['Accuracy', 'Precision', 'Recall', 'F1_Score']].std()

print("Mean of Evaluation Metrics:")
print(mean_metrics)
print("\nStandard Deviation of Evaluation Metrics:")
print(std_metrics)

Metric Value	Interpretation
* Accuracy	0.770 ± 0.014.	On average, the classifier correctly identified whether CA was 0 or 1 about 77 % ± 1.4% of the time. This is decent but may be misleading if there is class imbalance (e.g., many zeros).
* Precision	0.664 ±  0.032.	On average, of all the cases where the classifier predicted CA > 0, 66.4% ± 3.2% were correct. Upper moderate precision means some false positives (it sometimes predicts CA as 1 when the true value is 0).
* Recall 0.599 ± 0.036.	On average, the model only identified 59.9%  ± 3.6% of true CA > 0 cases .  So it's missing nearly a third of the true positives (false negatives are moderate).
* F1 Score	0.629 ± 0.023.	The harmonic mean of precision and recall. This moderate value indicates a trade-off between missing positives and over-predicting them.

In [ ]:
# create a plot of each metric by year:
fig, ax = plt.subplots(figsize=(10, 6))
for metric in ['Accuracy', 'Precision', 'Recall', 'F1_Score']:
    ax.plot(results_df['test_year'], results_df[metric], marker='o', label=metric)

ax.set_xlabel("Test Year")
ax.set_ylabel("Score")
ax.set_title("Walk-Forward Validation Metrics (2007–2022)")
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()

#### RF Classification Model Parameter Tuning

In [ ]:
# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 200], # The number of trees in the forest
    'max_depth': [None, 5, 10], # The maximum depth of the tree
    'min_samples_split': [2, 5], # The minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2] # The minimum number of samples required to be at a leaf node.
}

In [ ]:
results = []  # clear results list before starting

for i in range(2006, 2022):  # walk-forward from 2006 to 2021
    train_years = list(range(2001, i + 1))  # expanding window
    test_year = i + 1
    print(f"Training: {train_years[0]}–{train_years[-1]}, Testing: {test_year}")

    # Split train/test by year
    train_df = df_ts[df_ts['YEAR'].isin(train_years)]
    test_df = df_ts[df_ts['YEAR'] == test_year]

    # Define features and target
    features = ['PM25', 'NBA', 'PD', 'NAGRT', 'NNWVT', 'NBUT', 'ELEV',
              'NAGRT_lag1', 'NAGRT_lag2', 'NNWVT_lag1', 'NNWVT_lag2',
              'NBUT_lag1','NBUT_lag2']
    target = 'CA_bin'

    X_train = train_df[features]
    y_train = train_df[target]
    X_test = test_df[features]
    y_test = test_df[target]

    # Grid Search with 3-fold CV
    grid_search = GridSearchCV(
        estimator=RandomForestClassifier(random_state=42),
        param_grid=param_grid,
        scoring='recall',
        cv=3,
        n_jobs=-1,
        verbose=0
    )

    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Predict on test set
    y_pred = best_model.predict(X_test)

    # Evaluation metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    # Store results and best params
    results.append({
        'train_years': f"{train_years[0]}-{train_years[-1]}",
        'test_year': test_year,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1_Score': f1,
        'Best_Params': grid_search.best_params_
    })


In [ ]:
# convert results into a dataframe
resultspt_df = pd.DataFrame(results)

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
resultspt_df

In [ ]:
# Calculate and display the mean and standard deviation for each evaluation metric
mean_metrics = resultspt_df[['Accuracy', 'Precision', 'Recall', 'F1_Score']].mean()
std_metrics = resultspt_df[['Accuracy', 'Precision', 'Recall', 'F1_Score']].std()

print("Mean of Evaluation Metrics:")
print(mean_metrics)
print("\nStandard Deviation of Evaluation Metrics:")
print(std_metrics)

Metric Value	Interpretation
* Accuracy	0.773 ± 0.012.	On average, the classifier correctly identified whether CA was 0 or 1 about 77.3% ± 1.2% of the time. This is decent but may be misleading if there is class imbalance (e.g., many zeros).
* Precision	0.667 ±  0.039.	On average, of all the cases where the classifier predicted CA > 0, 66.4% ± 3.9% were correct. Upper moderate precision means some false positives (it sometimes predicts CA as 1 when the true value is 0).
* Recall 0.610 ± 0.037.	On average, the model only identified 61%  ± 3.7% of true CA > 0 cases .  So it's missing nearly a third of the true positives (false negatives are moderate).
* F1 Score	0.636 ± 0.026.	The harmonic mean of precision and recall. This moderate value indicates a trade-off between missing positives and over-predicting them.

Parameter tuning  improveed slightly model performance

In [ ]:
# create a plot of each metric by year:
fig, ax = plt.subplots(figsize=(10, 6))
for metric in ['Accuracy', 'Precision', 'Recall', 'F1_Score']:
    ax.plot(results_df['test_year'], resultspt_df[metric], marker='o', label=metric)

ax.set_xlabel("Test Year")
ax.set_ylabel("Score")
ax.set_title("Walk-Forward Validation Metrics (2007–2022) with Parameter Tuning")
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()

#### Training RF Classification Model on 2001–2021 and Predict 2022

In this section, for the SHAP analysis, we will train the RF classification model on data from 2001–2021 and predict, for 2022, whether the Normalized Asthma Mortality Rate (NAMR, represented by the variable CA) indicates nonoccurrence (0) or occurrence (1) of asthma mortality (binary classification)

In [ ]:
# Define features and target
features = ['PM25', 'NBA', 'PD', 'NAGRT', 'NNWVT', 'NBUT', 'ELEV',
            'NAGRT_lag1', 'NAGRT_lag2', 'NNWVT_lag1', 'NNWVT_lag2',
            'NBUT_lag1','NBUT_lag2']
target = 'CA_bin'

In [ ]:
# Training and test sets
train_df = df_ts[df_ts['YEAR'] <= 2021].dropna(subset=features + [target])
test_df = df_ts[df_ts['YEAR'] == 2022].dropna(subset=features + [target])

In [ ]:
# Define inputs
X_train = train_df[features]
y_train = train_df[target]

# Keep IDDPTO in test set
X_test_full = test_df[['IDDPTO'] + features+ ['CA_bin']].copy()
y_test = test_df[target].reset_index(drop=True)

In [ ]:
# get the tuned hiperpameter of the training 2001-2021
last_row_params = resultspt_df.iloc[-1]['Best_Params']
formatted_params = str(last_row_params).replace("'", "").replace(":", "=")
print(formatted_params)

In [ ]:
# Train model
clf = RandomForestClassifier(max_depth= None, min_samples_leaf= 2, min_samples_split= 2, n_estimators= 100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
# Predict using only feature columns
y_pred = clf.predict(X_test_full[features])

#  Add prediction to test set with IDDPTO
X_test_full['CA_bin_pred'] = y_pred

In [ ]:
# Evaluate prediction
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# Print metrics
print(f"Prediction Results for 2022:")
print(f"Accuracy:  {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")

Metric Value	Interpretation
* Accuracy	0.765:	The classifier correctly predicted whether CA was 0 or 1 in 2022 in about 76.5% of the time. This is decent but may be misleading if there is class imbalance (e.g., many zeros).
* Precision	0.669:	Of all the cases where the classifier predicted CA > 0 in 2022, 66.9 % were correct. This precision means a few false positives (it sometimes predicts CA as 1 when the true value is 0).
* Recall 0.582:	The model only identified 58.2%  of true CA > 0 cases.  So it's missing nearly a third of the true positives (false negatives are high).
* F1 Score	0.623:	The harmonic mean of precision and recall. This moderate value indicates a trade-off between missing positives and over-predicting them.

In [ ]:
# Preserve only IDDPTO, CA_bin, and CA_bin_pred from X_test_full
bin_pred_result = X_test_full[['IDDPTO', 'CA_bin', 'CA_bin_pred']].copy()

In [ ]:
# Display the new DataFrame
init_notebook_mode(all_interactive=True)
show(bin_pred_result)

#### SHAP (SHapley Additive Explanations)

In this section we will interpret the contribution of each independent variable to the final predition of the RF classification model using the SHAP method

In [ ]:
# SHAP for Binary Classifier
# Use TreeExplainer (optimized for RandomForest)
explainer = shap.TreeExplainer(clf)

In [ ]:
# get the test set
X = test_df[features]

In [ ]:
# Function to compute SHAP values for a single row
def compute_shap(row):
    return explainer.shap_values(row)

In [ ]:
# Parallel computation
shap_values_list = Parallel(n_jobs=-1)(
    delayed(compute_shap)(X.iloc[[i]]) for i in range(len(X))
)

In [ ]:
# Combine SHAP values for Class 1 into a single array
shap_values = np.vstack([vals[0][:, 1] for vals in shap_values_list])  # Class 1 SHAPs


In [ ]:
# Generate a SHAP summary plot to visualize the impact of features on the model's predictions
shap.summary_plot(shap_values, X, feature_names=features,
                  show= False, cmap = "plasma")
plt.title("(a) SHAP Summary Plot RF Classification", fontsize=17)
plt.xlabel("SHAP Values", size=12)
plt.ylabel("Features", size=12)
plt.ylabel("Features")
plt.show();

The SHAP summary plot shows that no predictor has a strong, consistent influence on the Random Forest classifier. Most SHAP values cluster tightly around zero, indicating weak contributions to predicting asthma mortality occurrence. Population density (PD) shows the widest spread, suggesting it has the most variable effect, while elevation (ELEV), NBUT, NNWVT, and their lagged versions have only small, mixed impacts. Overall, the model relies on many features with low predictive power, aligning with the modest classification performance observed.

###Part 2 – Regression Model


In this section, we will train and evaluate a Random Forest (RF) regression model to predict NAMR values where it is present  (CA_bin == 1 or NAMR values > 0). We will apply a walk-forward (expanding window) validation approach, which is appropriate for epidemiological studies involving time series data¹. We will start by training the RF regression model using data from 2001 to 2006 (a 5-year window) and testing it with data from 2007. The training window will then be expanded by one year at each iteration until it spans from 2001 to 2021, with 2022 data used for testing.

In [ ]:
# Filter bin_pred_result to keep only rows where CA_bin is 1
bin_positive = df_ts[(df_ts['CA_bin'] == 1)]

In [ ]:
# Display the filtered DataFrame
init_notebook_mode(all_interactive=True)
show(bin_positive)

### Training and Testing a RF Regression Model

In [ ]:
results = []  # store results

for i in range(2006, 2022):  # walk-forward from 2006 to 2021
    train_years = list(range(2001, i + 1))  # expanding window
    test_year = i + 1
    print(f"Training: {train_years[0]}–{train_years[-1]}, Testing: {test_year}")

    # Filter by year
    train_df = df_ts[df_ts['YEAR'].isin(train_years)]
    test_df = df_ts[df_ts['YEAR'] == test_year]

    # Filter by CA_bin == 1
    train_pos = train_df[train_df['CA_bin'] == 1].copy()
    test_pos = test_df[test_df['CA_bin'] == 1].copy()

    # Skip iteration if empty (avoid errors)
    if train_pos.empty or test_pos.empty:
        print(f"Skipped: No positive cases in train or test for {test_year}")
        continue

    # Features and regression target
    features = ['PM25', 'NBA', 'PD', 'NAGRT', 'NNWVT', 'NBUT', 'ELEV',
                'NAGRT_lag1', 'NAGRT_lag2', 'NNWVT_lag1', 'NNWVT_lag2',
                'NBUT_lag1','NBUT_lag2']
    target = 'CA'  # regression target

    X_train = train_pos[features]
    y_train = train_pos[target]
    X_test = test_pos[features]
    y_test = test_pos[target]

    # Train regression model with default hiperparameters
    model = RandomForestRegressor(n_jobs=-1, random_state=42)
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # Save results
    results.append({
        'train_years': f"{train_years[0]}-{train_years[-1]}",
        'test_year': test_year,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2
    })


In [ ]:
# convert the results into a data frame
results_df = pd.DataFrame(results)

In [ ]:
init_notebook_mode(all_interactive=True)
results_df

In [ ]:
# Calculate and display the mean and standard deviation for each evaluation metric
mean_metrics = results_df[['R2', "MAE", "RMSE"]].mean()
std_metrics = results_df[['R2', "MAE", "RMSE"]].std()

print("Mean of Evaluation Metrics:")
print(mean_metrics)
print("\nStandard Deviation of Evaluation Metrics:")
print(std_metrics)

The evaluation metrics indicate that the Random Forest regression model achieves moderate predictive performance, with a mean R² of 0.45  ± 0.13, suggesting it explains roughly half of the variance in asthma mortality across departments. The average errors are relatively low in magnitude (MAE ≈ 1.38  ± 0.17 deaths, RMSE ≈ 2.06  ±  0.33 deaths), but the gap between MAE and RMSE shows that some departments experience larger residual errors. The standard deviations—particularly for R² (~0.14)—suggest notable variability across folds, meaning the model’s performance is somewhat inconsistent depending on the training–testing split. Overall, the model captures meaningful patterns but shows instability and room for improvement, especially in predicting extreme mortality values or highly heterogeneous departments.

### RF Regression Model Parameter Tuning

In [ ]:
# Define feature list and hyperparameter grid
features = ['PM25', 'NBA', 'PD', 'NAGRT', 'NNWVT', 'NBUT', 'ELEV',
                'NAGRT_lag1', 'NAGRT_lag2', 'NNWVT_lag1', 'NNWVT_lag2',
                'NBUT_lag1','NBUT_lag2']

param_grid = {
    'n_estimators': [100, 200], # The number of trees in the forest
    'max_depth': [None, 10], # The maximum depth of the tree. I
    'min_samples_leaf': [1, 2] # The minimum number of samples required to be at a leaf node
}

In [ ]:
results = []

for i in range(2006, 2022):  # walk-forward from 2006 to 2021
    train_years = list(range(2001, i + 1))
    test_year = i + 1
    print(f"Training: {train_years[0]}–{train_years[-1]}, Testing: {test_year}")

    # Filter by year
    train_df = df_ts[df_ts['YEAR'].isin(train_years)]
    test_df = df_ts[df_ts['YEAR'] == test_year]

    # Filter to CA_bin == 1
    train_pos = train_df[train_df['CA_bin'] == 1].copy()
    test_pos = test_df[test_df['CA_bin'] == 1].copy()

    # Skip if empty
    if train_pos.empty or test_pos.empty:
        print(f"Skipped: No positive cases in train or test for {test_year}")
        continue

    X_train = train_pos[features]
    y_train = train_pos['CA']
    X_test = test_pos[features]
    y_test = test_pos['CA']

    # Grid search for best model
    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(random_state=42),
        param_grid=param_grid,
        scoring='r2',
        cv=3,
        n_jobs=-1,
        verbose=0
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_

    # Predict and evaluate
    y_pred = best_model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    # Store result
    results.append({
        'train_years': f"{train_years[0]}-{train_years[-1]}",
        'test_year': test_year,
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Best_Params': grid_search.best_params_
    })

In [ ]:
# convert results into a dataframe
resultspt_df = pd.DataFrame(results)

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
resultspt_df

In [ ]:
# Calculate and display the mean and standard deviation for each evaluation metric
mean_metrics = results_df[['R2', "MAE", "RMSE"]].mean()
std_metrics = results_df[['R2', "MAE", "RMSE"]].std()

print("Mean of Evaluation Metrics:")
print(mean_metrics)
print("\nStandard Deviation of Evaluation Metrics:")
print(std_metrics)

On average, parameter tuning did not improve the model’s performance. However, for the 2001–2021 training period, it did lead to better results. Therefore, we will use the tuned parameters in the next section.

### Training RF Regression Model on 2001–2021 and Predict 2022

In this section, for the SHAP analysis and mapping, we will train the RF regression model on data from 2001–2021 and predict the 2022 Normalized Asthma Mortality Rate in departments where it is present (CA_bin = 1 or NAMR > 0).

In [ ]:
# Define features and target
features = ['PM25', 'NBA', 'PD', 'NAGRT', 'NNWVT', 'NBUT', 'ELEV',
                'NAGRT_lag1', 'NAGRT_lag2', 'NNWVT_lag1', 'NNWVT_lag2',
                'NBUT_lag1','NBUT_lag2']
target = 'CA'

In [ ]:
# Prepare training and test sets
train_df = df_ts[df_ts['YEAR'] <= 2021].dropna(subset=features + [target])
test_df = df_ts[df_ts['YEAR'] == 2022].dropna(subset=features + [target])

In [ ]:
# Filter for positive cases (CA_bin == 1)
train_pos = train_df[train_df['CA_bin'] == 1].copy()
test_pos = test_df[test_df['CA_bin'] == 1].copy()

In [ ]:
# Define inputs
X_train = train_pos[features]
y_train = train_pos[target]

In [ ]:
# Keep IDDPTO and make prediction-ready copy
X_test_full = test_pos[['IDDPTO'] + features + [target]].copy()
X_test = X_test_full[features]
y_test = test_pos[target].reset_index(drop=True)

In [ ]:
# get the tuned hiperpameter of the training 2001-2021
last_row_params = resultspt_df.iloc[-1]['Best_Params']
formatted_params = str(last_row_params).replace("'", "").replace(":", "=")
print(formatted_params)

In [ ]:
# Train regression model
reg = RandomForestRegressor(max_depth= 10, min_samples_leaf= 2, n_estimators= 200, n_jobs=-1, random_state=42)
reg.fit(X_train, y_train)

In [ ]:
# Predict using feature columns
y_pred = reg.predict(X_test)

In [ ]:
# Store prediction in column CA_pred
X_test_full['CA_pred'] = y_pred

In [ ]:
# Evaluate
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
# Print evaluation
print("Prediction Results for 2022:")
print(f"R²: {r2:.3f}, MAE: {mae:.3f}, RMSE: {rmse:.3f}")

Metric Value	Interpretation:

The Random Forest regression model trained only on positive asthma mortality
cases (CA_bin = 1) achieved a  R² of 0.636. The model explains about 63.6% of the variability in asthma mortality among departments where mortality actually occurred. This indicates good predictive capacity in estimating asthma mortality rates in 2022.

In [ ]:
# select IDDPTO, CA and CA_pred from X_test_full
reg_pred_result = X_test_full[['IDDPTO', 'CA', 'CA_pred']].copy()

In [ ]:
# visualize the reults
init_notebook_mode(all_interactive=True)
reg_pred_result

In [ ]:
# create a scatterplot between CA_2022 vs CA_2022_Pred
plt.figure(figsize=(10, 6))
plt.scatter(reg_pred_result['CA'], reg_pred_result['CA_pred'])
plt.xlabel('CA_2022 (Actual)')
plt.ylabel('CA_2022_Pred (Predicted)')
plt.title('Scatter Plot of Actual vs. Predicted Mortality Rates (2022)')
plt.grid(True)
plt.show()

Although the model underestimates high mortality and overestimates low mortality, the RF regression model trained on NAMR > 0 captures the true distribution of asthma mortality more accurately than the model trained on the full dataset.

### SHAP (SHapley Additive Explanations)

In this section we will interpret the contribution of each independent variable to the final predition of the RF regression model using the SHAP method

In [ ]:
# SHAP for Regression
# Use TreeExplainer (optimized for RandomForest)
explainer = shap.TreeExplainer(reg)

# get test set
X = test_pos[features]

# Function to compute SHAP values for a single row
def compute_shap(row):
    # For regression, shap_values returns a single 1D array
    return explainer.shap_values(row)

# Parallel computation (adjust n_jobs as needed)
shap_values_list = Parallel(n_jobs=-1)(
    delayed(compute_shap)(X.iloc[[i]]) for i in range(len(X))
)

# Stack the 1D SHAP value arrays vertically
# No need to index [:, 1] as there's only one set of SHAP values for regression
shap_values = np.vstack(shap_values_list)

In [ ]:
# Generate a SHAP summary plot to visualize the impact of features on the model's predictions
shap.summary_plot(shap_values, X, feature_names=features,
                  show= False,  cmap = "plasma")
plt.title("(b) SHAP Summary Plot RF Regression", fontsize=17)
plt.xlabel("SHAP Values", size=12)
plt.ylabel("Features", size=12)
plt.show();

We observe low PD values were associated with higher predicted NAMR in the regression model. This may reflect limited access to healthcare services and larger emergency response times in sparsely populated areas [3]. Rural areas often have limited access to healthcare facilities and professionals, which can delay diagnosis and treatment, leading to higher mortality rates[4]. However, it could also be influenced by the ‘small counts’ issue common in epidemiological studies, where low population numbers lead to unstable or unreliable rate estimates. [5]

## 🌍 Mapping actual vs predicted asthma mortality rate

As explained above, since our dataset contains a high proportion of zero values, we will map the predicted NAMR for 2022 only in departments where the actual values are greater than zero. Departments with an actual NAMR value of zero will retain a predicted value of zero.

### Preparing data for mapping

In [ ]:
# from df_ts get filter 2022 and CA_bin equal to 0
dpto_zero = df_ts[df_ts['YEAR'] == 2022]
dpto_zero = dpto_zero[dpto_zero['CA_bin'] == 0]

In [ ]:
# keep IDDPTO and CA_bin columns
dpto_zero = dpto_zero[['IDDPTO', 'CA_bin']]

In [ ]:
# rename CA_bin to CA and copy the column as CA_pred
dpto_zero.rename(columns={'CA_bin': 'CA'}, inplace=True)
dpto_zero['CA_pred'] = dpto_zero['CA'].copy()

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
show(dpto_zero)

In [ ]:
# get df info
dpto_zero.info()

In [ ]:
# concat reg_pred_result with dpto_zero
df_concat = pd.concat([reg_pred_result, dpto_zero])

In [ ]:
# get df info
df_concat.info()

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
show(df_concat)

In [ ]:
# Load dataset with data per department
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/data.gpkg")

In [ ]:
# keep IDDPTO and geometry of the gdf
dpto_geom = gdf[['IDDPTO', 'geometry']]

In [ ]:
# merge df_concat and dpto_geom by IDDPTO preserve  df_concat data
df_map = pd.merge(df_concat, dpto_geom, on='IDDPTO', how='left')

In [ ]:
# rename CA as CA_2022 and CA_pred as CA_2022_PRED
df_map.rename(columns={'CA': 'CA_2022', 'CA_pred': 'CA_2022_PRED'}, inplace=True)

In [ ]:
# round CA_2022_PRED to two decimal places
df_map['CA_2022_PRED'] = df_map['CA_2022_PRED'].round(2)

In [ ]:
# convert df_map to a gdf
gdf_map = gpd.GeoDataFrame(df_map, geometry='geometry')

In [ ]:
# visualize geodataframe
init_notebook_mode(all_interactive=True)
show(gdf_map)

In [ ]:
# save df_map as gpkg file
gdf_map.to_file("pdt/asthma_mortality/data/gpkg/results_RF.gpkg", driver="GPKG")

### Calculate classification schema for mapping

We will use [Pysal](https://pysal.org/)'s [mapclassify](https://pysal.org/mapclassify/index.html) library to determine the best classifier for the choropleth map.

We will use the map classifier with the best ACDM (mean Absolute Deviation Around the class Median). In Pysal, ACDM refers to the mean absolute deviation around the class median. It is a measure of a classifier's fit to the data, specifically by evaluating the average distance between each data point and the median value of the assigned class.

In [ ]:
# open results_RF.gpkg as a gdf
df_cl = gpd.read_file("pdt/asthma_mortality/data/gpkg/results_RF.gpkg")

In [ ]:
# visualize the dataframe
init_notebook_mode(all_interactive=True)
show(df_cl)

In [ ]:
# get df basic info
df_cl.info()

In [ ]:
# Get the length of the dataframe 'df_cl'
len(df_cl)

In [ ]:
# Select data to analize
selected_data = df_cl.loc[:,["CA_2022", "CA_2022_PRED"]]

In [ ]:
# Classify the data into 4 quantile groups
q4 = mapclassify.Quantiles(selected_data, k=4)
q4

In [ ]:
# Equal Interval Classification
ei5 = mapclassify.EqualInterval(selected_data, k=5)
ei5

In [ ]:
# Classify the data into groups based on the head/tail breaks algorithm
ht = mapclassify.HeadTailBreaks(selected_data)
ht

In [ ]:
# MaximumBreaks classification method
mb5 = mapclassify.MaximumBreaks(selected_data, k=5)
mb5

In [ ]:
# Apply the Standard Deviation and Mean classification method to the selected data.
msd = mapclassify.StdMean(selected_data)
msd

In [ ]:
# Apply Fisher-Jenks classification with 5 classes
fj5 = mapclassify.FisherJenks(selected_data, k=5)
fj5

ACDM(mean Absolute Deviation Around the class Median) visualization

In [ ]:
# Bunch classifier objects
class5 = q4, ei5, ht, mb5, msd, fj5
# Collect ADCM for each classifier
fits = np.array([c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms["classifier"] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ["ADCM", "Classifier"]
ax = sns.barplot(
    y="Classifier", x="ADCM", data=adcms, hue= adcms["Classifier"],  legend=False
)
plt.show();

### Create choropleth maps

Two classifiers have the lowest ACDM: FisherJenks and HeadTailBreaks. We'll select FisherJenks as the classifier to create the choropleth maps.

In [ ]:
# Convert the bins to a list for further processing
bins = fj5.bins.tolist()
bins

In [ ]:
# insert 0 at 0 position
bins.insert(0, 0.0)
bins

We will use the bins calculated during the EDA of the target variable using the full time series (2001–2022), as this approach captures the entire variability of the target.

In [ ]:
# bins from EDA of the target variable NAMR by department (choropleth map)
bins = [0.0, 1.15, 3.83, 7.97, 14.81, 39.02]

In [ ]:
# Create a custom classification using UserDefined for actual values
classi_actual = mapclassify.UserDefined(df_cl["CA_2022"], bins)

In [ ]:
# Create a custom classification using UserDefined for predicted values
classi_pred = mapclassify.UserDefined(df_cl["CA_2022_PRED"], bins)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 8))
fig.subplots_adjust(hspace=0, wspace=-0.9)
plt.suptitle('Normalized Asthma Mortality Rate 2022', fontsize=14, y=1)

# Plot classi_actual
classi_actual.plot(
    df_cl,
    legend=False,  # We'll build it manually
    axis_on=False,
    border_color='black',
    cmap="viridis_r",
    ax=axes[0]
)

# Plot classi_pred
classi_pred.plot(
    df_cl,
    legend=False,  # We'll build it manually
    axis_on=False,
    border_color='black',
    cmap="viridis_r",
    ax=axes[1]
)


# Custom bin labels and colors
#bin_labels = ["0.00", "0.00-0.65", "0.65-2.16", "2.16-4.13", "4.13-7.49", "7.49-14.15"]
bin_labels = ["0.00", "0-1.15", "1.15-3.83", "3.83-7.97", "7.97-14.81", "14.81-39.02"]
n_bins = len(bin_labels)
# cmap = mpl.cm.get_cmap("viridis_r", n_bins)
cmap = mpl.colormaps.get_cmap("viridis_r").resampled(n_bins)
colors = [mpl.colors.to_hex(cmap(i)) for i in range(cmap.N)]

# Create legend patches for bins
bin_patches = [Patch(facecolor=color, edgecolor='black', label=label)
               for color, label in zip(colors, bin_labels)]


# Combine all patches
all_patches = bin_patches

# Display custom legend
#axes[0].legend(handles=all_patches, loc='upper right', bbox_to_anchor=(1.1, 0.4), fontsize=8)
axes[1].legend(handles=all_patches, loc='upper right', bbox_to_anchor=(0.9, 0.25), fontsize=10)

# Set titles
axes[0].set_title('Actual', fontsize=12)
axes[1].set_title('Predicted', fontsize=12)

plt.tight_layout()
plt.show();

## 📚 References

[1] Hasanah, S., S. Abdullah, and Dian Lestari. "Bayesian method for hurdle regression." ICSA-Int. Conf. Stat. Anal. Vol. 2019. 2019.

[2] Utku, A., & Akcayol, M. A. (2024). Spread patterns of COVID-19 in European countries: hybrid deep learning model for prediction and transmission analysis. Neural Computing and Applications, 36(17), 10201-10217.

[3] J. E. Garrett, “Health service accessibility and deaths
from asthma,” Thorax, vol. 52, no. 3, pp. 205–206, 1997,
doi: 10.1136/thx.52.3.205

[4] Thompson, J. A., Mudaranthakam, D. P., &#38; Chollet-Hinton, L. (2023). The rural mortality penalty in U.S. hospital patients with COVID-19. <i>Research Square</i>. https://doi.org/10.21203/rs.3.rs-3467683/v1>

[5] W. S. D. of Health, “Guidelines for using confidence
intervals for public health assessment,” 2012. [Online].
Available: https://doh.wa.gov/sites/default/files/legacy/Documents/1
500/ConfIntGuide.pdf




